## Simple CNN

In this assignment, you need to build a simple CNN to classify images in CIFAR-10 dataset.

![cifar10](https://pytorch.org/tutorials/_images/cifar10.png)

The framework is given below. What you need to do is to build your net. Then tune the hyperparameters, train the NN, and achieve **60%+** accuracy. (about 1h training on CPU is enough)

You can also refer to the [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm

print(torch.__version__)

Some hyperparameters. They are maybe not the best, you need to tune, especially the learning rate.

In [ ]:
LEARNING_RATE = 1e-3
NUM_EPOCHS = 20
BATCH_SIZE = 32 # Mini-batch size
DEVICE = torch.device('cuda')
CONFIG = {"lr": LEARNING_RATE,
          "num_epochs": NUM_EPOCHS,
          "batch_size": BATCH_SIZE}

**TODO**: Define your NN here.

Some reference networks:
* LeNet
* AlexNet
* VGG
* GoogleLeNet (maybe to large for your PC to train)
* ResNet (too large)

Ref: [Overview of CNN Development - Zhihu](https://zhuanlan.zhihu.com/p/66215918)

Do not directly copy others' code. Check these networks' structure and write code yourself.

In [ ]:
class Net(nn.Module):
    """
    TODO: Implementation of a simple Convolutional neural network.
    HINT: You can refer to several famous CNNs, like LeNet5, VGG16, ResNet
        Actually you cannot directly copy the code in the demo,
        since you need to recalculate the shape of the tensor.
    """
    """YOUR CODE HERE"""
    def __init__(self):
        super(Net, self).__init__()
        pass # your code
        
    def forward(self, x):
        pass # yuor code
    """END OF YOUR CODE"""

model = net().to(device=DEVICE)
print(model)

Load CIFAR-10 dataset. Downloading will be started automatically.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # magic numbers
                         std=[0.229, 0.224, 0.225]) # 3 channels
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Define the loss function.

In [ ]:
criterion = F.cross_entropy

Define the optimizer. You can **choose other** optimizers as you like.

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

Evaluation process.

In [ ]:
def evaluate(model_eval, loader_eval, criterion_eval):
    model_eval.eval()
    loss_eval = 0
    correct = 0.
    pbar = tqdm(total = len(loader_eval), desc='Evaluation', ncols=100)
    with torch.no_grad():
        for data, target in loader_eval:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            loss_eval += criterion_eval(output, target).item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            pbar.update(1)
    pbar.close()

    loss_eval = loss_eval / loader_eval.dataset.__len__()
    accuracy = correct / loader_eval.dataset.__len__()
    response = {'loss': loss_eval, 'acc': accuracy}
    return response

The main training function defined below. Be careful of [overfitting](https://en.wikipedia.org/wiki/Overfitting)!

In [ ]:
train_acc = np.zeros(NUM_EPOCHS)
eval_acc = np.zeros(NUM_EPOCHS)
train_loss = np.zeros(NUM_EPOCHS)
eval_loss = np.zeros(NUM_EPOCHS)

model.train()
for epoch_idx in range(NUM_EPOCHS):
    pbar = tqdm(total = len(train_dataloader), desc='Train - Epoch {}'.format(epoch_idx), ncols=100)
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.to(DEVICE), target.to(DEVICE)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        pbar.update(1)
    pbar.close()

    train_resp = evaluate(model, train_dataloader, criterion)
    eval_resp = evaluate(model, test_dataloader, criterion)

    print ('-*-*-*-*-*- Epoch {} -*-*-*-*-*-'.format(epoch_idx))
    print ('Train Loss: {:.6f}\t'.format(train_resp['loss']))
    print ('Train Acc: {:.6f}\t'.format(train_resp['acc']))
    print ('Eval Loss: {:.6f}\t'.format(eval_resp['loss']))
    print ('Eval Acc: {:.6f}\t'.format(eval_resp['acc']))
    print ('\n')

    train_acc[epoch_idx] = train_resp['acc']
    eval_acc[epoch_idx] = eval_resp['acc']
    train_loss[epoch_idx] = train_resp['loss']
    eval_loss[epoch_idx] = eval_resp['loss']

    # save model and training data
    torch.save(model, 'simple-cnn.pth'.format(CONFIG["network"]))
    np.savez('simple-cnn',config=CONFIG,train_acc=train_acc, eval_acc=eval_acc,train_loss=train_loss, eval_loss=eval_loss)

Visualize the training process

In [ ]:
import matplotlib.pyplot as plt

train_acc = np.zeros(NUM_EPOCHS)
eval_acc = np.zeros(NUM_EPOCHS)
train_loss = np.zeros(NUM_EPOCHS)
eval_loss = np.zeros(NUM_EPOCHS)

def plot_acc(train_acc,eval_acc):
    plt.plot(train_acc,label="Train")
    plt.plot(eval_acc,label="Test")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

def plot_loss(train_loss,eval_loss):
    plt.plot(train_loss,label="Train")
    plt.plot(eval_loss,label="Test")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
plot_acc(train_acc,eval_acc)
plot_loss(train_loss,eval_loss)